In [2]:
# Import public packages and functions
import os
import pandas as pd
import numpy as np
import sys
import json
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp
import pickle

# inserting the lib folder to the compiler
sys.path.insert(0, './lib')
sys.path.insert(0, './utils/')

#import utils_plotting
import utils_misc

from lib_ECoG import ECoG
from lib_event import EVENTS
from lib_data import DATA_IO

In [3]:
PATH_CURR   = os.path.abspath(os.curdir)    # current code
PATH        = (str(Path(PATH_CURR).parent)) # data repository: upper directory where datasets situated
SUB_LIST    = utils_misc.get_SUB_list(DATA_IO.path_data) # get the SUB id list which we have a recording of them
event_files = utils_misc.get_files_with_specific_format(DATA_IO.path_events, suffix=".pkl")

# 1. Extract ECoG Events

In [3]:
for SUB in SUB_LIST:

    if(SUB[0] == "0"): # patients labelled with 0 beginning have ECoG recordings
        
        ECoG_SUB           = ECoG(PATH, SUB)
        
        # 1. Load event history of patient
        df_lfp_events      = pd.read_csv(DATA_IO.path_events + "SUB_"+ SUB +"_EVENTS_LFP.csv")
        
        # remove short events
        df_lfp_events      = df_lfp_events[df_lfp_events.duration>=0.2]
    
        # removing irregular dyskinesia scores
        df_lfp_events      = df_lfp_events[(~df_lfp_events['dyskinesia_arm'].isnull()) & (~df_lfp_events['dyskinesia_total'].isnull())]
    
        # removing uncontinious events due to the artifact removal 
        df_lfp_events      = df_lfp_events[df_lfp_events.duration== (df_lfp_events.event_finish_index-df_lfp_events.event_start_index)/2048]

        # 2. Get the LFP recordings dataframe
        df_ecog_recordings = ECoG_SUB.extract_ECoG_events_segments(df_lfp_events)
        
        # 3. Save the dataframe as a pickle file
        df_ecog_recordings.to_pickle(DATA_IO.path_events + "SUB_"+ SUB +"_EVENTS_ECoG_RECORDINGS.pkl")
        

ECoG Recording: SUB-008
---> left hemisphere ECoG recording found...
... pickle loading: C:\Users\a.kaymak\Desktop\Papers\2025 Parkinson STN-DBS Dyskinesia LFP-ECG\files\data\sub-008\008_mergedData_v4.0_ecog_left.P
... SUB - 008 : 07-06 channel was not found!
... SUB - 008 : 08-07 channel was not found!
... SUB - 008 : 09-08 channel was not found!
... SUB - 008 : 10-09 channel was not found!
... SUB - 008 : 11-10 channel was not found!
... SUB - 008 : 12-11 channel was not found!
ECoG Recording: SUB-009
---> left hemisphere ECoG recording found...
... pickle loading: C:\Users\a.kaymak\Desktop\Papers\2025 Parkinson STN-DBS Dyskinesia LFP-ECG\files\data\sub-009\009_mergedData_v4.0_ecog_left.P
... SUB - 009 : 07-06 channel was not found!
... SUB - 009 : 08-07 channel was not found!
... SUB - 009 : 09-08 channel was not found!
... SUB - 009 : 10-09 channel was not found!
... SUB - 009 : 11-10 channel was not found!
... SUB - 009 : 12-11 channel was not found!
ECoG Recording: SUB-010
... pi

# 2. Merge ECoG Event Files

In [3]:
df_ecog_events           = pd.DataFrame()
for file in event_files:
    if("ECoG" in file):
        if(len(df_ecog_events)==0):
            df_ecog_events = pd.read_pickle(DATA_IO.path_events + file)
        else:
            df_ecog_events = pd.concat([df_ecog_events, pd.read_pickle(DATA_IO.path_events + file)], ignore_index=True)


In [4]:
df_ecog_events.to_pickle(DATA_IO.path_events + "ECoG_EVENTS.pkl")

# 3. Save ECoG-Baseline Recordings

In [6]:
for SUB in SUB_LIST:  

    if((SUB[0]=="0") & (SUB!="019")): # patients labelled with 0 beginning have ECoG recordings
        
        ECoG_SUB      = ECoG(PATH, SUB)
        ECoG_baseline = ECoG_SUB.get_baseline_recording(t_min=0, t_max=5)

        # pickle the baseline dictionary
        with open(DATA_IO.path_events + "baseline_recordings/ECoG/"+ SUB +".pkl", 'wb') as handle:
            pickle.dump(ECoG_baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

ECoG Recording: SUB-008
---> left hemisphere ECoG recording found...
... pickle loading: C:\Users\a.kaymak\Desktop\Papers\2025 Parkinson STN-DBS Dyskinesia LFP-ECG\files\data\sub-008\008_mergedData_v4.0_ecog_left.P
... SUB - 008 : 07-06 channel was not found!
... SUB - 008 : 08-07 channel was not found!
... SUB - 008 : 09-08 channel was not found!
... SUB - 008 : 10-09 channel was not found!
... SUB - 008 : 11-10 channel was not found!
... SUB - 008 : 12-11 channel was not found!
... SUB - 008 : right_02-01 channel was not found!
... SUB - 008 : right_03-02 channel was not found!
... SUB - 008 : right_04-03 channel was not found!
... SUB - 008 : right_05-04 channel was not found!
... SUB - 008 : right_06-05 channel was not found!
... SUB - 008 : right_07-06 channel was not found!
... SUB - 008 : right_08-07 channel was not found!
... SUB - 008 : right_09-08 channel was not found!
... SUB - 008 : right_10-09 channel was not found!
... SUB - 008 : right_11-10 channel was not found!
... S